In [1]:
import torch
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")

# Check PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

PyTorch version: 2.4.1
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [2]:
import torch

# Set the device
device = "mps" if torch.backends.mps.is_available() else "cpu"

# Create data and send it to the device
x = torch.rand(size=(3, 4)).to(device)
x

tensor([[0.3383, 0.4070, 0.4463, 0.3100],
        [0.6431, 0.2256, 0.5896, 0.4187],
        [0.4121, 0.7701, 0.4010, 0.2637]], device='mps:0')

In [3]:
import torch
from torchvision import datasets, transforms

import torch.nn as nn
import torch.optim as optim

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Failed to download (trying next):
<urlopen error [Errno 61] Connection refused>



100%|██████████| 9912422/9912422 [00:18<00:00, 549240.87it/s] 


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 61] Connection refused>



100%|██████████| 28881/28881 [00:00<00:00, 115329.76it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 61] Connection refused>



100%|██████████| 1648877/1648877 [00:12<00:00, 128839.56it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 61] Connection refused>



100%|██████████| 4542/4542 [00:00<00:00, 1321026.89it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Step [100/938], Loss: 0.2237
Epoch [1/5], Step [200/938], Loss: 0.0634
Epoch [1/5], Step [300/938], Loss: 0.0872
Epoch [1/5], Step [400/938], Loss: 0.2053
Epoch [1/5], Step [500/938], Loss: 0.0445
Epoch [1/5], Step [600/938], Loss: 0.0867
Epoch [1/5], Step [700/938], Loss: 0.1681
Epoch [1/5], Step [800/938], Loss: 0.1513
Epoch [1/5], Step [900/938], Loss: 0.0291
Epoch [2/5], Step [100/938], Loss: 0.0107
Epoch [2/5], Step [200/938], Loss: 0.0242
Epoch [2/5], Step [300/938], Loss: 0.0363
Epoch [2/5], Step [400/938], Loss: 0.0299
Epoch [2/5], Step [500/938], Loss: 0.0083
Epoch [2/5], Step [600/938], Loss: 0.0560
Epoch [2/5], Step [700/938], Loss: 0.0250
Epoch [2/5], Step [800/938], Loss: 0.0137
Epoch [2/5], Step [900/938], Loss: 0.0023
Epoch [3/5], Step [100/938], Loss: 0.0439
Epoch [3/5], Step [200/938], Loss: 0.0040
Epoch [3/5], Step [300/938], Loss: 0.0137
Epoch [3/5], Step [400/938], Loss: 0.0250
E

In [4]:
from sklearn.metrics import accuracy_score, f1_score

# Load the test dataset
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Evaluate the model
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(target.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9880
F1 Score: 0.9880
